# Νευρωνικά Δίκτυα με PyTorch

Τα δίκτυα βαθιάς μάθησης τείνουν να είναι τεράστια με δεκάδες ή εκατοντάδες επίπεδα, για αυτό και ο όρος «βαθιά». Μπορείτε να δημιουργήσετε ένα τέτοιο βαθύ δίκτυο χρησιμοποιώντας μόνο πίνακες με βάρη όπως κάναμε στην προηγούμενη ενότητα, αλλά γενικά είναι πολύ πολύπλοκο και δύσκολο στην εφαρμογή του. Η PyTorch διαθέτει ένα βολικό module `nn` που παρέχει έναν ωραίο τρόπο για την αποτελεσματική υλοποίηση μεγάλων νευρωνικών δικτύων.


In [ ]:
# Import των απαραίτητων βιλιοθηκών

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import helper

import matplotlib.pyplot as plt

Τώρα θα υλοποιήσουμε ένα μεγαλύτερο δίκτυο που μπορεί να λύσει ένα (πρώην) δύσκολο πρόβλημα, αναγνωρίζοντας χαρακτήρες κειμένου απο μια εικόνα. Στο πρόβλημα αυτό θα χρησιμοποιήσουμε τη βάση δεδομένων MNIST που αποτελείται από χειρόγραφα ψηφία σε αποχρώσεις του γκρι. Κάθε εικόνα είναι 28x28 pixel, και μπορείτε να δείτε ένα δείγμα παρακάτω

<img src='assets/mnist.png'>

Στόχος μας είναι να δημιουργήσουμε ένα νευρωνικό δίκτυο που μπορεί να δεκετεί σαν είσοδο μία από αυτές τις εικόνες και να προβλέψει το ψηφίο της εικόνας.

Πρώτα από όλα, πρέπει να κατεβάσουμε το σύνολο των δεδομένων μας. Αυτό γίνεται διαθέσιμο μέσω του package `torchvision`. Ο παρακάτω κώδικας θα κατεβάσει όλη τη βάση δεδομένων MNIST, και στη συνέχεια θα δημιουργήσει σύνολα δεδομένων εκπαίδευσης και ελέγχου για εμάς. Μην ανησυχείτε πάρα πολύ για τις λεπτομέρειες εδώ, θα μάθετε περισσότερα για αυτό αργότερα.


In [ ]:
### Εκτελέστε τον κώδικα σε αυτό το κελί

from torchvision import datasets, transforms

# Καθορίζω ένα μετασχηματισμό (transform) για κανονικοποίηση των δεδομένων
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Κατε΄βαζω και φορτώνω τα δεδομένα εκπαίδευσης
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Έχουμε τα δεδομένα εκπαίδευσης φορτωμένα στον `trainloader` και δημιουργούμε έναν iterator με το `iter(trainloader)`. Αργότερα, θα το χρησιμοποιήσουμε σε ένα επαναληπτικό βρόχο για το σύνολο των δεδομένων εκπαίδευσης, όπως παρακάτω

```python
for image, label in trainloader:
    ## θα εκτελέσουμε διάφορες εντολές πανω στις εικόνας (images) και τις ετικέτες (labels)
```

Θα παρατηρήσετε ότι δημιούργησα το `trainloader` με μέγεθος batch 64 και `shuffle=True`. Το μέγεθος του batch είναι ο αριθμός των εικόνων που χρησιμοποιούμε σε μια επαναληπτική διαδικασία από τον φορτωτή δεδομένων και τα εισάγουμε στο δίκτυό μας, το υποσύνολο αυτών των εικόνων ονομάζεται *batch*. Το `shuffle=true` ανακατεύει το σύνολο δεδομένων κάθε φορά που ξεκινάμε ξανά το φορτωτή δεδομένων. Στη περίπτωση εδω θα φορτώσω ένα batch, για να ελέγξουμε και να δουμε τα δεδομένα. Μπορούμε να δούμε παρακάτω ότι το `images` είναι απλώς ένας τανυστής με μέγεθος `(64, 1, 28, 28)`. Που σημαίνει, 64 εικόνες ανά batch, 1 κανάλι χρώματος και 28x28 ανάλυση εικόνας.


In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

Έτσι μοιάζει μία από τις εικόνες.

In [ ]:
plt.imshow(images[1].numpy().squeeze(), cmap='Greys_r');

Αρχικά, ας προσπαθήσουμε να δημιουργήσουμε ένα απλό δίκτυο για αυτό το σύνολο δεδομένων χρησιμοποιώντας πίνακες βάρους και πολλαπλασιασμούς πινάκων. Στη συνέχεια, θα δούμε πώς να το κάνουμε χρησιμοποιώντας το module `nn` της PyTorch η οποία παρέχει μια πολύ πιο βολική και ισχυρή μέθοδο για τον ορισμό των αρχιτεκτονικών δικτύου.

Τα δίκτυα που εχουμε δει μέχρι τώρα ονομάζονται *πλήρως συνδεδεμένα* (fully-connected) ή *πυκνά* (dense) δίκτυα. Κάθε κόμβος σε ένα επίπεδο συνδέεται με κάθε κόμβο στο επόμενο επίπεδο. Στα πλήρως συνδεδεμένα δίκτυα, η είσοδος σε κάθε επίπεδο πρέπει να ένα μονοδιάστατο διάνυσμα (το οποίο μπορεί να στοιβαχθεί σε ένα δισδιάστατο τανυστή ως μία παρτίδα (batch) πολλαπλών δειγμάτων). Ωστόσο, οι εικόνες μας είναι 28x28 2D tensors, οπότε πρέπει να τις μετατρέψουμε σε 1D διανύσματα. Μιλώντας για μεγέθη, πρέπει να μετατρέψουμε το σύνολο των εικόνων που έχουν μέγεθος "(64, 1, 28, 28)" σε μεγεθος "(64, 784)", το 784 είναι 28 φορές το 28. Αυτό ονομάζεται συνήθως *ισοπέδωση* (flattening), μετασχηματίζωντας τις 2D εικόνες σε μία επίπεδη αναπαράσταση ενος μονοδιάστατου διανύσματος.

Στα προηγούμενα παραδείγματα δημιουργήσατε ένα δίκτυο με έναν κόμβο εξόδου. Εδώ χρειαζόμαστε 10 κόμβους εξόδου, έναν για κάθε ψηφίο. Θέλουμε το δίκτυό μας να προβλέψει το ψηφίο που εμφανίζεται σε μια εικόνα, οπότε αυτό που θα κάνουμε είναι να υπολογίσουμε τις πιθανότητες ότι η εικόνα απεικονίζει ένα απο τα ψηφία. Αυτό καταλήγει σε μία διακριτή κατανομή πιθανότητας για τις κλάσεις (ψηφία) δίνοντας την πιο πιθανή κλάση για την εικόνα. Για το λόγο αυτό χρειαζόμαστε 10 κόμβους εξόδου για τις 10 κλάσεις (ψηφία). Στη συνέχεια, θα δούμε πώς μετατρέπουμε την έξοδο του δικτύου σε μία κατανομή πιθανότητας.


> **Άσκηση:** Μετασχηματίστε σε επίπεδη αναπαράσταση τη παρτίδα των εικόνων `images`. Στη συνέχεια, δημιουργήστε ένα δίκτυο πολλαπλών επιπέδων με 784 κόμβους εισόδου, 256 κρυφούς κόμβους και 10 κόμβους εξόδου χρησιμοποιώντας τυχαίους τανυστές για τα βάρη και τις πολώσεις. Προς το παρόν, χρησιμοποιήστε τη sigmoid για συνάρτηση ενεργοποίησης στο κρυφό επίπεδο. Αφήστε το επίπεδο εξόδου χωρίς συνάρτηση ενεργοποίησης, θα προσθέσουμε στη συνέχεια μία συνάρηση που θα μας δίνει μια κατανομή πιθανότητας.

In [ ]:
## Λύση
def activation(x):
    return 1/(1+torch.exp(-x))

# Επίπεδη αναπαράσταση των εικόνων εισόδου
inputs = images.view(images.shape[0], -1)

# Δημιουργία παραμέτρων
w1 = torch.randn(784, 256)
b1 = torch.randn(256)

w2 = torch.randn(256, 10)
b2 = torch.randn(10)

h = activation(torch.mm(inputs, w1) + b1)

out = torch.mm(h, w2) + b2

Τώρα έχουμε 10 εξόδους για το δίκτυό μας. Θέλουμε να δώσουμε μια εικόνα στο δίκτυό μας και να υπολογίσουμε μια κατανομή πιθανότητας για τις κλάσεις οι οποίες θα ορίζουν τις ποιο πιθανές κλασεις στις οποίες ανήκει η εικόνα. Κάτι που να μοιάζει με αυτό:
<img src='assets/image_distribution.png' width=500px>

Εδώ βλέπουμε ότι η πιθανότητα για κάθε τάξη είναι περίπου η ίδια. Αυτό αντιπροσωπεύει ένα μη εκπαιδευμένο δίκτυο, δεν έχει δει ακόμα δεδομένα (δεν έχει εκπαιδευτεί), οπότε απλώς επιστρέφει μια ομοιόμορφη κατανομή με ίσες πιθανότητες για κάθε κλάση.

Για τον υπολογισμό αυτής της κατανομής πιθανότητας, χρησιμοποιούμε συχνά τη [**softmax** function](https://en.wikipedia.org/wiki/Softmax_function). Η εξίσωσή της είναι αυτή:

$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$

Αυτό που κάνει είναι να περιορίζει κάθε είσοδο $x_i$ μεταξύ 0 και 1 και να κανονικοποιεί τις τιμές για να πάρουμε μια σωστή κατανομή πιθανότητας όπου όλες οι πιθανότητες αθροίζονται στη τιμή 1.

> **Άσκηση:** Υλοποιήστε τη συνάρτηση `softmax` που υπολογίζει και επιστρέφει κατανομές πιθανότητας για κάθε δείγμα μίας παρτίδας δεδομένων εκπαίδευσης. Σημειώστε ότι θα πρέπει να δώσετε προσοχή στα μεγέθη των τανυστών όταν το κάνετε αυτό. Εάν έχετε έναν τανυστή `a` με μέγεθος `(64, 10)` και έναν τανυστή `b` με μέγεθος `(64,)`, η πράξη `a/b` θα σας επιστρέψει σφάλμα επειδή η PyTorch θα προσπαθήσει να κάνει τη διαίρεση στις στήλες (broadcasting) και θα επιστρέψει μια αναντιστοιχία μεγέθους. Ο τρόπος σκέψης για αυτό είναι για κάθε ένα από τα 64 παραδείγματα, θέλετε μόνο να διαιρέσετε με μια τιμή, το άθροισμα του παρονομαστή. Επομένως, το `b` πρέπει να έχει μέγεθος `(64, 1)`. Με τον τρόπο αυτό, η PyTorch θα διαιρέσει τις 10 τιμές σε κάθε γραμμή του `a` με τη μία τιμή της κάθε γραμμής του `b`. Δώστε επίσης προσοχή στο πώς υπολογίζετε το άθροισμα. Θα πρέπει να ορίσετε την `dim` παράμετρο στο `torch.sum`. Η τιμή `dim=0` υπολογίζει το άθροισμα στις γραμμές ενώ το`dim=1` υπολογίζει το άθροισμα στις στήλες.


In [ ]:
## Λύση
def softmax(x):
    return torch.exp(x)/torch.sum(torch.exp(x), dim=1).view(-1, 1)

probabilities = softmax(out)

# Έχει το σωστό μέθεθος? Πρέπει να είναι (64, 10)
print(probabilities.shape)
# Αθροίζονται στη τιμή ένα?
print(probabilities.sum(dim=1))

## Υλοποιώντας δίκτυα με PyTorch

Η PyTorch παρέχει ένα module το `nn` που κάνει την υλοποίηση πολύ απλούστερη. Εδώ θα σας δείξω πώς να υλοποίησετε το ίδιο δίκτυο όπως το προηγούμενο με 784 εισόδους, 256 κρυφούς κόμβους, και 10 κόμβους εξόδου και έξοδο softmax.

In [ ]:
from torch import nn

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Η είσοδοι στο κρυφό επ΄ίπεδο γραμμικού μετασχηματισμού
        self.hidden = nn.Linear(784, 256)
        # Επίπεδο εξόδου, 10 κόμβοι - ένας για κάθε ψηφίο
        self.output = nn.Linear(256, 10)
        
        # Οριζω τη συναρτηση ενεργοποίησης sigmoid και την συνάρτηση ενεργοποίησης εξόδου softmax 
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # Εισάγω τον τανυστή εισόδου σε κάθε μία απο τις απαραίτητες πράξεις
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x

Ας δούμε τί γίνεται βήμα βήμα.

```python
class Network(nn.Module):
```

Δημιουργία	κλάσης	με κληρονομικότητα απο το `nn.Module`. Σε συνδιασμό με το `super().__init__()` δημιουργεί μια κλάση που παρακολουθεί την αρχιτεκτονική και παρέχει πολλές χρήσιμες μεθόδους και χαρακτηριστικά. Είναι υποχρεωτικό να κληρονομήσω απο το `nn.Module` όταν φτιάχνω μία κλάση για το δίκτυό μου. Το όνομα της ίδιας της κλάσης μπορεί να είναι οτιδήποτε.

```python
self.hidden = nn.Linear(784, 256)
```

Αυτή η γραμμή δημιουργεί ένα module για το γραμμικό μετασχηματισμό, $x\mathbf{W} + b$, με 784 εισόδους και 256 εξόδους και τα εκχωρεί στο `self.hidden`. Το module αυτόματα δημιουργεί τους πίνακες (τανυστές) weight και bias τους οποίους θα χρησιμοποιήσουμε στο μέθοδο `forward`. Μπορείτε να αποκτήσετε πρόσβαση στους τανυστές αυτούς μόλις το δίκτυο (`net`) δημιουργηθεί με τη βοήθεια των `net.hidden.weight` και `net.hidden.bias`.

```python
self.output = nn.Linear(256, 10)
```

Ομοίως, αυτή τη γραμμή δημιουργεί έναν άλλο γραμμικό μετασχηματισμό με 256 εισόδους και 10 εξόδους.

```python
self.sigmoid = nn.Sigmoid()
self.softmax = nn.Softmax(dim=1)
```

Εδώ ορίζω την λειτουργία (operation) για τη συναρτηση ενεργοποίησης sigmoid και την softmax για την έξοδο. Ορίζοντας `dim=1` στο `nn.Softmax(dim=1)` υπολογίζω το softmax στις στήλες.

```python
def forward(self, x):
```

Τα δίκτυα στη PyTorch που δημιουργούνται απο τη κλάση `nn.Module` πρέπει να εχουν μία μέθοδο `forward` καθορισμένη. Δέχονται σαν είσοδο εναν τανυστή `x` και τον προοθούν μέσα απο τις λειτουργίες που έχουμε ορίσει στη μέθοδο `__init__`.

```python
x = self.hidden(x)
x = self.sigmoid(x)
x = self.output(x)
x = self.softmax(x)
```

Εδώ ο τανυστής εισόδου `x` περνάει μέσα απο κάθε μία λειτουργία και επανεκχωρείτε ως `x`. Βλέπουμε οτι ο τανυστής εισόδου περναέι μέσα απο το κρυφό επίπεδο, μετά απο τη σιγμοειδή συνάρτηση, μετά απο το επίπεδο εξόδου και τέλος απο τη συναρτηση  softmax. Δεν έχει σημασία πως θα ονομάσουμε τις μεταβλητές, παρα μόνο να ταιριάζουν οι είσοδοι και οι έξοδοι με τν αρχιτεκτονική του δικτύου μας. Δεν έχει σημασία και η σειρά με την οποία ορίζουμε τα επίπεδα στη μέθοδο `__init__`, αλλά πρέπει να περνάμε σωστά τα δεδομένα μας απο κάθε επίπεδο με τη σωστή σειρά στη μέθοδο `forward`.

Τωρα μπορούμε να δημιουργήσουμε το αντικείμενο (object) `Network`.

In [ ]:
# Δημιουργώ το δίκτυο και βλέπω την αρχιτεκτονική του
model = Network()
model

Μπορείτε να ορίσετε το δίκτυο κάπως πιο συνοπτικά και απλά χρησιμοποιώντας το module `torch.nn.functional`. Αυτός είναι ένας πιο συνηθισμένος τρόπος που ορίζονται τα δίκτυα, καθώς πολλές πράξεις τους είναι απλές συναρτήσεις μεταβλητών. Συνήθως εισάγουμε αυτό το module ως `F`, `import torch.nn.functional as F`.

In [ ]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Είσοδοι για το κρυφό επίπεδο γραμμικού μετασχηματισμού
        self.hidden = nn.Linear(784, 256)
        # Επίπεδο εξόδου, 10 κόμβοι - ένας για κάθε ψηφίο
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        # Κρυφά επιπεδα με σιγμοειδή συνάρτηση ενεργοποίησης
        x = F.sigmoid(self.hidden(x))
        # Επίπεδο εξόδου με συνάρτηση ενεργοποίησης softmax 
        x = F.softmax(self.output(x), dim=1)
        
        return x

### Συναρτήσεις Ενεργοποίησης

Μέχρι τώρα είδαμε μόνο την ενεργοποίηση softmax, αλλά γενικά οποιαδήποτε συνάρτηση μπορεί να χρησιμοποιηθεί ως συνάρτηση ενεργοποίησης. Η μόνη απαίτηση είναι ότι για ένα δίκτυο που προσεγγίζει μια μη γραμμική συνάρτηση, οι λειτουργίες ενεργοποίησης πρέπει να είναι και αυτές μη γραμμικές.
Ακολουθούν μερικά ακόμη παραδείγματα κοινών συναρτήσεων ενεργοποίησης: Tanh (hyperbolic tangent), και ReLU (rectified linear unit).

<img src="assets/activation.png" width=700px>

Στην πράξη, η συνάρτηση ReLU χρησιμοποιείται σχεδόν αποκλειστικά ως συνάρτηση ενεργοποίησης για τα κρυφά επίπεδα.

### Ένα πιο πολύπλοκο δίκτυο

<img src="assets/mlp_mnist.png" width=600px>

> **Άσκηση:** Δημιουργείστε ένα δίκτυο με 784 κόμβους εισόδου, ένα κρυφό επίπεδο με 128 κόμβους και συνάρτηση ενεργοποίησης ReLU, μετά ένα κρυφό επίπεδο με 64 κόμβους και ReLU συνάρτηση ενεργοποίησης, και τέλος ένα επίπεδο εξόδου με συνάρτηση ενεργοποίησης softmax activation όπως παραπάνω. Μπορείτε να χρησιμοποιείσετε τη ενεργοποίηση ReLU χρησιμοποιώντας τη μέθοδο `nn.ReLU` ή τη συνάρτηση `F.relu`.

Είναι καλή πρακτική να ονομάζετε τα επίπεδα σας ανάλογα με τον τύπο του δικτύου τους, για παράδειγμα 'fc' αντιστοιχεί σε fully-connected επίπεδο. Οπότε μπορω να χρησιμοποιήσω, `fc1`, `fc2`, και `fc3` ώς τα ονόματα για τα επίπεδά μου.

In [ ]:
## Λύση

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Ορίζω τα επίπεδα, 128, 64, 10 κόμβους αντίστοιχα
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        # Επίπεδο εξόδου, 10 κόμβοι - ένας για κάθε ψηφίο
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        ''' Πρόσω τροφοδότηση σε όλο το δίκτυα, επιστρέφει τις εξόδους των εικόνων-ψηφίων '''
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        
        return x

model = Network()
model

### Αρχικοποίηση weights και biases

Τα βάρη και η πόλωση αρχικοποιούνται αυτόματα για εσάς, αλλά μπορείτε να προσαρμόσετε τον τρόπο με τον οποίο αρχικοποιούνται. Τα βάρη και η πόλωση είναι τανυστές συνδεδεμένοι με το επίπεδο που έχετε ορίσει, μπορείτε να τα δείτε εκτελώντας `model.fc1.weight`.

In [ ]:
print(model.fc1.weight)
print(model.fc1.bias)

Για μή αυτόματη ενεργοποίηση, πρέπει να επέμβουμε στους ίδους τους τανυστές. Στην ουσία είναι autograd *μεταβλητές*, οπότε πρέπει πρώτα να πάρουμε τις μεταβλητές αυτές με `model.fc1.weight.data`. Όταν πάρουμε τους τανυστές, μπορούμε να τους γεμίσουμε με μηδενικά (για τις πολώσεις) ή με τυχαίες κανονικές τιμές.

In [ ]:
# ορίζω ολες τις πολώσες ως μηδέν
model.fc1.bias.data.fill_(0)

In [ ]:
# κανω δειγματοληψία απο μία τυχαία κανονική κατανομη με τυπική απόκλιση = 0.01
model.fc1.weight.data.normal_(std=0.01)

### Πρόσω τροφοδότηση

Τώρα που έχουμε ένα δίκτυο, ας δούμε τι συμβαίνει όταν περνάμε σε αυτό μια εικόνα.

In [ ]:
# Φόρτωσε μερικά δεδομένα 
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Αλλαγή μεγέθους εικόνας σε μονοδιάστατο διάνυσμα, το νέο μέγεθος είναι (batch size, color channels, image pixels) 
images.resize_(64, 1, 784)
# ή images.resize_(images.shape[0], 1, 784) για να πάρετε αυτόματα το μέγεθος του batch

# Τροφοδότηση του δικτύου
img_idx = 0
ps = model.forward(images[img_idx,:])

img = images[img_idx]
helper.view_classify(img.view(1, 28, 28), ps)

Όπως βλέπετε παραπάνω, το δίκτυό μας ουσιαστικά δεν έχει ιδέα για το ποιο είναι αυτό το ψηφίο. Αυτό οφείλετε στο ότι ακόμα δεν έχει εκπαιδευτεί το δίκτυο, και όλα τα βάρη είναι τυχαία!

### Χρήση του `nn.Sequential`

Η PyTorch μας δίνει ενα ακόμα πιο βολικό και ευκολο τρόπο να δηιουργήσουμε τα δίκτυά μας όπως παρακάτω, όπου ο τανυστής διέρχεται διαδοχικά μεσω των απαραίτιτων πράξεων, `nn.Sequential` ([documentation](https://pytorch.org/docs/master/nn.html#torch.nn.Sequential)). Χρησμιποιώντας αυτό το τρόπο, το ίδιο δίκτυο μπορει να δημιουργηθεί οπως παρακάτω:

In [ ]:
# Οι υπερπαράμετροι του δικτύου μας
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Δημιουργώ ενα feed-forward δίκτυο
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

# Πρόσω τροφοδότηση σε όλο το δίκτυο και απεικόνηση της εξόδου
images, labels = next(iter(trainloader))
images.resize_(images.shape[0], 1, 784)
ps = model.forward(images[0,:])
helper.view_classify(images[0].view(1, 28, 28), ps)

Οι πράξεις είναι διαθέσιμες δίνοντας τον κατάλληλο δείκτη. Για παράδειγμα, εάν θέλετε να εκτελέσετε το πρώτο γραμμικό μετασχηματισμό και να δείτε τα βάρη, εκτελείτε το `model[0]`.

In [ ]:
print(model[0])
model[0].weight

Μπορείτε επίσης να ορίσετε `OrderedDict` για να δώσετε όνομα σε κάθε ξεχωριστό επίπεδο και στις λειτουργίες του, αντί ενός αυξητικού ακέραιου δείκτη. Σημειώστε ότι τα ονόματα πρέπει να είναι μοναδικά, επομένως _κάθε πράξη πρέπει να έχει μοναδικό όνομα_.

In [ ]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('softmax', nn.Softmax(dim=1))]))
model

Τώρα μπορείτε να έχετε πρόσβαση στα επίδεδα είτε με τον αριθμό τους είτει με το όνομά τους

In [ ]:
print(model[0])
print(model.fc1)

Στο επόμενο notebook, θα δούμε πως μπορούμε να εκπαιδεύσουμε αυτό το νευρωνικό δίκτυο για να βρίσκει με ακρίβεια σε ποιο ψηφίo αντιστοιχεί κάθε εικόνα του MNIST dataset.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

Αυτό το notebook 📖 δημιουργήθηκε για το μάθημα ***Υπολογιστική Νοημοσύνη και Μηχανική Μάθηση*** του Τμήματος Μηχανικών Παραγωγής και Διοίκησης, της Πολυτεχνικής Σχολής του Δημοκριτείου Πανεπιστημίου Θράκης.<br>
This notebook is made available under the Creative Commons Attribution [(CC-BY)](https://creativecommons.org/licenses/by/4.0/legalcode) license. Code is also made available under the [MIT License](https://opensource.org/licenses/MIT).<br>
Author: Asst. Prof. Angelos Amanatiadis
<img src="assets/cc.png" style="width:55px; float: right; margin: 0px 0px 0px 0px;"></img>
<img src="assets/mit.png" style="width:40px; float: right; margin: 0px 10px 0px 0px;"></img>